In [1]:
# Znak wykrzyknika (!) pozwala wywołać komendy systemowe Linuxa
!git clone https://github.com/WebCrawlerr/GAT.git


Cloning into 'GAT'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 38 (delta 12), reused 35 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 28.47 KiB | 7.12 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
# Pobranie zmian
!git pull origin main


fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [3]:
# Wyświetla listę plików w pobranym folderze
!ls -R  GAT

GAT:
main.py  requirements.txt  src	tests

GAT/src:
config.py	    dataset.py	 model.py     train.py
data_processing.py  features.py  __pycache__  utils.py

GAT/src/__pycache__:
config.cpython-312.pyc		 features.cpython-312.pyc
data_processing.cpython-312.pyc  train.cpython-312.pyc
dataset.cpython-312.pyc		 utils.cpython-312.pyc

GAT/tests:
test_features.py  test_pipeline.py


In [4]:
import shutil
import os
import sys
# 1. Zmiana katalogu roboczego (odpowiednik komendy 'cd')
os.chdir('./GAT')
print(f"Aktualny katalog: {os.getcwd()}")

Aktualny katalog: /kaggle/working/GAT


In [5]:
# Najpierw sprawdźmy wersję po restarcie (czasem GPU ma starszego PyTorcha)
import torch
version = torch.__version__
print(f"PyTorch version: {version}")

# Instalacja PyG i bibliotek pomocniczych (korzystamy z gotowych paczek .whl, żeby nie kompilować przez 20 min)
# UWAGA: Ta komenda dynamicznie dobiera link do wersji Twojego PyTorcha
torch_version_suffix = version.split('+')[0].replace('.', '') # np. 260 lub 240
cuda_version_suffix = 'cu' + torch.version.cuda.replace('.', '') # np. cu121

print(f"Instaluję dla: torch-{torch_version_suffix} + {cuda_version_suffix}")

# Instalacja zależności
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv \
  -f https://data.pyg.org/whl/torch-{version}.html

# Instalacja reszty z Twojego pliku (bez ponownego instalowania torch-geometric, bo zrobiliśmy to wyżej)
!pip install -r requirements.txt

PyTorch version: 2.6.0+cu124
Instaluję dla: torch-260 + cu124
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 125.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:

In [6]:
import torch
import torch_geometric

print(f"CUDA dostępne: {torch.cuda.is_available()}")
print(f"Wersja PyTorch: {torch.__version__}")
print(f"Wersja PyG: {torch_geometric.__version__}")

# Próba importu kluczowych warstw
try:
    from torch_geometric.nn import GATConv
    print("✅ GATConv zaimportowany pomyślnie!")
except ImportError as e:
    print(f"❌ Błąd importu GATConv: {e}")

# Sprawdzenie czy mamy backendy (scatter/sparse)
# PyG 2.7+ potrafi czasem działać bez nich (wolniej), ale warto sprawdzić
try:
    import torch_scatter
    print("✅ torch_scatter obecny")
except ImportError:
    print("⚠️ torch_scatter BRAK (Może to spowolnić trening lub wywołać błąd)")

try:
    import torch_sparse
    print("✅ torch_sparse obecny")
except ImportError:
    print("⚠️ torch_sparse BRAK (Wymagany dla niektórych operatorów)")

CUDA dostępne: True
Wersja PyTorch: 2.6.0+cu124
Wersja PyG: 2.7.0
✅ GATConv zaimportowany pomyślnie!
✅ torch_scatter obecny
✅ torch_sparse obecny


In [7]:
!python main.py --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed

Starting GAT BRD4 Binding Prediction Pipeline...
Loading and filtering data from /kaggle/input/bindingdb-smiles/BindingDB_All.tsv in chunks of 100000...
Processed 1000000 rows...
Processed 2000000 rows...
Processed 3000000 rows...
Finished processing 3078912 rows.
Found 22758 records for BRD4.
Cleaning and Labeling...
Final dataset size: 22616
Creating Graph Dataset in ./data/processed (this may take a while)...
Processing...
Processing Molecules:  11%|█▋             | 2576/22616 [00:03<00:27, 727.12it/s][10:21:45] Can't kekulize mol.  Unkekulized atoms: 1 4 6
[10:21:45] Can't kekulize mol.  Unkekulized atoms: 18 19 23
[10:21:45] Can't kekulize mol.  Unkekulized atoms: 23 24 28
[10:21:45] Can't kekulize mol.  Unkekulized atoms: 1 4 6
[10:21:45] Can't kekulize mol.  Unkekulized atoms: 20 21 25
[10:21:45] Can't kekulize mol.  Unkekulized atoms: 20 21 25
[10:21:45] Can't kekulize mol.  Unkekulized atoms: 1 4 6
[10:21:45] Can't kekulize mol.  Unkekulized atoms: 1 4 6
[10:21:45] Can't kekul

In [8]:
import torch
import pandas as pd
import os
from IPython.display import FileLink
import shutil

# 1. Zdefiniuj nazwę pliku, w którym skrypt zapisał model
# (Musisz sprawdzić w kodzie main.py jak nazwano plik, zazwyczaj to 'model.pth' lub 'checkpoint.pt')
# Poniżej zakładam nazwę 'checkpoint.pt', zmień ją jeśli Twój skrypt używa innej!
model_filename = 'checkpoint.pt' 

# Sprawdźmy co masz w folderze working
print("Pliki w folderze roboczym:", os.listdir('.'))

# 2. Zapiszmy wyniki metryk do pliku CSV (żeby nie przepadły)
results = {
    'Metric': ['AUC', 'AP', 'F1'],
    'Value': [0.8654, 0.9311, 0.8700]
}
df_results = pd.read_csv('training_metrics.csv') if os.path.exists('training_metrics.csv') else pd.DataFrame(results)
df_results.to_csv('final_results.csv', index=False)
print("Zapisano wyniki do final_results.csv")

# 3. Pakowanie wszystkiego do ZIP (Model + Wyniki)
output_filename = 'GAT_BRD4_Results.zip'
files_to_zip = ['final_results.csv']

# Dodaj model do zip tylko jeśli istnieje
if os.path.exists(model_filename):
    files_to_zip.append(model_filename)
else:
    # Spróbujmy znaleźć jakikolwiek plik .pth lub .pt
    for f in os.listdir('.'):
        if f.endswith('.pth') or f.endswith('.pt'):
            files_to_zip.append(f)

# Komenda systemowa do zipowania
cmd = f"zip {output_filename} {' '.join(files_to_zip)}"
os.system(cmd)

print(f"📦 Utworzono archiwum: {output_filename}")

# 4. Generowanie linku do pobrania
FileLink(output_filename)

Pliki w folderze roboczym: ['main.py', 'training_curves.png', 'models', 'src', 'requirements.txt', '.gitignore', 'data', '.git', 'tests', 'confusion_matrix.png']
Zapisano wyniki do final_results.csv
  adding: final_results.csv (deflated 5%)
📦 Utworzono archiwum: GAT_BRD4_Results.zip


/kaggle/working/GAT/GAT_BRD4_Results.zip